<a href="https://colab.research.google.com/github/gacerioni/redis-workshop-json-search-vs/blob/master/redis_workshop_vector_intro_pt_br_gabs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Workshop - Redis como VectorDB - INTRO (TEM OUTROS!)

## Vector Searches & Large Language Models

![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)


Bem-vind[ao]s ao Workshop! Vamos ter uma experiência hands-on sobre alguns temas centrais do Redis, bem além do Caching.


Para uma experiência premium, como a que eu quero que vocês tenham, recomendo MUITO utilizar o Redis Insight (App ou Web) pra apoiar na visualização dos dados.

https://redis.com/redis-enterprise/redis-insight/

---

Novamente, vamos direto ao ponto. Para pegar o fio da meada, passando pela introdução, veja este outro notebook [aqui](https://colab.research.google.com/github/gacerioni/redis-workshop-json-search-vs/blob/master/redis-workshop-vector-similarity-search.ipynb).

---

## Objetivos do Workshop

Este Notebook é uma pequena demonstração do Redis como um Vector DB. Depois, vamos ver uma implementação de RAG e Semantic/LLM Caching.


Espero que gostem! 🖖


## Conceito - Bancos de dados de vetores

Os dados são frequentemente não estruturados, o que significa que não são descritos por um esquema bem definido. Exemplos de dados não estruturados incluem trechos de texto, imagens, vídeos ou áudio. Uma abordagem para armazenar e pesquisar dados não estruturados é usar embeddings de vetores.

**O que são vetores?**\
Em aprendizado de máquina e IA, vetores são sequências de números que representam dados. Eles são as entradas e saídas dos modelos, encapsulando informações subjacentes em uma forma numérica. Vetores transformam dados não estruturados, como textos, imagens, vídeos e áudios, em um formato que os modelos de aprendizado de máquina podem processar.

**Por que eles são importantes?**\
Vetores capturam padrões complexos e significados semânticos inerentes aos dados, tornando-os ferramentas poderosas para uma variedade de aplicações. Eles permitem que modelos de aprendizado de máquina compreendam e manipulem dados não estruturados de forma mais eficaz.

**Melhorando a busca tradicional.**\
A busca tradicional por palavras-chave ou lexical depende de correspondências exatas de palavras ou frases, o que pode ser limitante. Em contraste, a busca vetorial, ou busca semântica, aproveita a rica informação capturada nos embeddings de vetores. Ao mapear dados em um espaço vetorial, itens semelhantes são posicionados próximos uns dos outros com base em seu significado. Essa abordagem permite resultados de busca mais precisos e significativos, pois considera o contexto e o conteúdo semântico da consulta, e não apenas as palavras exatas usadas.

# Passo 1 - Criar uma conta Free no Redis Cloud

Basta seguir o passo a passo [aqui](https://colab.research.google.com/github/gacerioni/redis-workshop-notebook-validator/blob/master/redis-workshop-setup-notebook-validator.ipynb)!

# Passo 2 - Setup Rápido

## Instalaçao das libs do Python e redis-cli

In [2]:
# Instale as deps, como redis, sentence transformers, etc
# equivale a
# pip install redis pandas sentence-transformers tabulate numpy requests
!pip install -r https://raw.githubusercontent.com/gacerioni/redis-workshop-json-search-vs/master/deps/vector-intro/requirements.txt

# E instalar a CLI, via redis-tools, que inclui a famosa redis-cli
!apt-get update
!apt-get install -y redis-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 35.9 MB/s eta 0:00:00
  Using cached redis-5.0.7-py3-none-any.whl (252 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 96.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 MB 12.0 MB/s eta 0:00:00
  Using cached sentence_transformers-3.0.1-py3-none-any.whl (227 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 58.1 MB/

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
^C
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
redis-tools is already the newest version (5:6.0.16-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 46 not upgraded.


# Iniciando os trabalhos - All hands on deck!

## Conectando com o Redis server

In [1]:
import os

# Coloque aqui os dados do seu DB do Redis Cloud
REDIS_HOST="redis-18884.c98.us-east-1-4.ec2.redns.redis-cloud.com"
REDIS_PORT=18884
REDIS_PASSWORD="lgZgS90vZJpnS4F2Y5EJ97YJTFGUUdvF"

# Caso o SSL esteja ativo pro endpoint, adicione --tls
# Recomendo não misturar lé com cré aqui, visto que não vamos ter nenhuma informação sensível passando pelo fio.
if REDIS_PASSWORD!="":
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT} -a {REDIS_PASSWORD} --no-auth-warning"
else:
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT}"

# Caso o SSL esteja ativo pro endpoint, use rediss:// como o URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"
INDEX_NAME = f"qna:idx"

# Teste a Redis connection
!redis-cli $REDIS_CONN PING

PONG


In [2]:
# Testando via Python (redis-py)
import redis
redis = redis.Redis(
  host=REDIS_HOST,
  port=REDIS_PORT,
  password=REDIS_PASSWORD)
redis.ping()

True

## 1 - Importando e preparando as libs que iremos usar

Este primeiro bloco vai garantir que todas as dependências estejam prontas pra gente brincar com o lab.

In [3]:
import json
import time

import numpy as np
import pandas as pd
import requests
import redis
from redis.commands.search.field import (
    NumericField,
    TagField,
    TextField,
    VectorField,
)
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query
from sentence_transformers import SentenceTransformer

redis = redis.Redis(
  host=REDIS_HOST,
  port=REDIS_PORT,
  password=REDIS_PASSWORD,
  decode_responses=True)


redis.ping()

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


True


## 2 - Carregando a massa de dados - Bikes

Vamos ingerir uma pequena massa de dados que contém bikes e suas descrições... como um SKU da vida.





In [5]:
URL = "https://raw.githubusercontent.com/gacerioni/redis-workshop-json-search-vs/master/deps/vector-intro/data/bikes.json"
response = requests.get(URL, timeout=10)
bikes = response.json()

# vamos ver o que foi carregado
json.dumps(bikes[0], indent=2)

'{\n  "model": "Jigger",\n  "brand": "Velorim",\n  "price": 270,\n  "type": "Bicicletas Infantis",\n  "specs": {\n    "material": "alum\\u00ednio",\n    "weight": "10"\n  },\n  "description": "Pequena e poderosa, a Jigger \\u00e9 a melhor pedalada para os menores ciclistas! Esta \\u00e9 a menor bicicleta infantil com pedal no mercado dispon\\u00edvel sem freio de contrapedal, a Jigger \\u00e9 a escolha ideal para o raro pequeno tenaz pronto para partir. Dizemos raro porque esta pequena bicicleta n\\u00e3o \\u00e9 ideal para um iniciante nervoso, mas \\u00e9 um verdadeiro galope para um verdadeiro velocista. A Jigger \\u00e9 uma bicicleta infantil leve de 12 polegadas e atender\\u00e1 \\u00e0 necessidade de velocidade do seu pequeno. \\u00c9 uma bicicleta de uma \\u00fanica velocidade que torna o aprendizado a pedalar simples e intuitivo. Ela at\\u00e9 tem uma al\\u00e7a na parte inferior do selim para que voc\\u00ea possa ajudar seu filho facilmente durante o treinamento! A Jigger est\

## 3 - Carregar os dados no Redis como JSON - Binary Tree (não é uma String)

Vamos carregar essas bikes como documentos no Redis. Documentos JSON, claro!

Vamos usar o conceito de pipeline, que pode ser muito útil com volumes maiores de dados. O Redis Cloud tem um proxy zero-latency aqui pra cuidar do multiplexing.

In [7]:
pipeline = redis.pipeline()
for i, bike in enumerate(bikes, start=1):
    redis_key = f"bikes:{i:03}"
    pipeline.json().set(redis_key, "$", bike)
res = pipeline.execute()

Com os dados carregados, podemos pegar trechos do documento JSON dessa maneira:

In [9]:
res = redis.json().get("bikes:010", "$.model")
print(res)

['Summit']


## 3 - Escolha um modelo de embedding que entenda o Português Brasileiro

A **HuggingFace** possui um extenso catálogo de modelos de **embedding** de texto que podem ser servidos localmente através do framework **SentenceTransformers**.

Os gringos costumam usar o modelo MS MARCO, amplamente utilizado em mecanismos de busca, chatbots e outras aplicações de IA.

Entretanto, quero que este lab funcione com o nosso lindo idioma. Vamos fazer com `paraphrase-multilingual-MiniLM-L12-v2`

In [10]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## 4 - Hora do Show: Gerando os embeddings no Redis!

O primeiro passo aqui é iterar e selecionar as chaves que vamos trabalhar. Neste caso, são as `bikes::`

In [12]:
keys = sorted(redis.keys("bikes:*"))

Agora, use as chaves como entrada para o comando `JSON.MGET`, juntamente com o campo `$.description`, para coletar as descrições em uma lista.

Em seguida, passe a lista de descrições para o método `.encode()`:

In [14]:
descriptions = redis.json().mget(keys, "$.description")

descriptions = [item for sublist in descriptions for item in sublist]

embedder = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

embeddings = embedder.encode(descriptions).astype(np.float32).tolist()

VECTOR_DIMENSION = len(embeddings[0])

print("Gabs says: This ST creates embeddings with {0} dimensions. Redis MUST know that. :D".format(VECTOR_DIMENSION))

Gabs says: This ST creates embeddings with 384 dimensions. Redis MUST know that. :D


Finalmente, insira as descrições vetorizadas nos documentos de bicicletas no Redis usando o comando `JSON.SET`.

O seguinte comando insere um novo campo em cada um dos documentos sob o **JSONPath** `$.description_embeddings`.

Mais uma vez, faça isso usando um **pipeline** para evitar viagens desnecessárias pela rede:

In [17]:
#print(keys/embeddings)

pipeline = redis.pipeline()
for key, embedding in zip(keys, embeddings):
    pipeline.json().set(key, "$.description_embeddings", embedding)
pipeline.execute()

[True, True, True, True, True, True, True, True, True, True, True]

Vai ficar meio poluído aqui... mas olhem só como o dado está no Redis.

Podem usar o RedisInsight também!

In [18]:
res = redis.json().get("bikes:010")
print(res)

{'model': 'Summit', 'brand': 'nHill', 'price': 1200, 'type': 'Bicicleta de Montanha', 'specs': {'material': 'liga', 'weight': '11.3'}, 'description': 'Esta bicicleta de montanha econômica da nHill tem um bom desempenho tanto em ciclovias quanto em trilhas. O garfo com 100mm de curso absorve terrenos acidentados. Pneus largos Kenda Booster oferecem aderência em curvas e trilhas molhadas. A transmissão Shimano Tourney oferece engrenagens suficientes para encontrar um ritmo confortável para subir, e os freios a disco hidráulicos Tektro freiam suavemente. Se você quer uma bicicleta acessível que pode levar para o trabalho, mas também levar para trilhas nos fins de semana, ou está apenas atrás de uma pedalada estável e confortável na ciclovia, a Summit oferece um bom custo-benefício.', 'description_embeddings': [0.028230920433998108, 0.1364552080631256, -0.06720077246427536, -0.009281334467232227, -0.0297839492559433, -0.010982556268572807, 0.011184461414813995, 0.04645169526338577, -0.0727